In [1]:
import pandas as pd
import nltk

In [2]:
#libreria tokenizador
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mcm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#libreria palabras vacias
from nltk.corpus import stopwords
nltk.download("stopwords")
palabras_vacias=set(stopwords.words("spanish"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mcm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import es_core_news_sm
import spacy
lematiza = es_core_news_sm.load()

In [5]:
#Steamming
from nltk import SnowballStemmer

In [6]:
df_Cardi=pd.read_csv('cardiUnificado.csv', encoding='utf-8')
textos=df_Cardi['texto']

In [7]:
df_lema=pd.DataFrame({'texto': []})
df_Stemmer=pd.DataFrame({'texto': []})

In [8]:
lineaLeida=0
cammel=0
log=1
cadenalema=''

for texto in textos:
    lineaLeida+=1
    if(lineaLeida<=log):
        print("---------------------------------- Linea "+ str(lineaLeida) +" ---------------------------------- ")
        #eliminar menciones de @user
        print('--Elimina @user')
    texto=re.sub('@user',"",texto)
    #Eliminar caracteres especiales: + / ? ! ¡ ...
    if(lineaLeida<=log):
        print('--Elimina caracteres especiales [\.$\*¿\?¡!\+\-\[\]\)\(\\\]"," ')
    texto=re.sub("[\.$\*¿\?¡!\+\-\[\]\)\(\\\]","",texto)
    #Para los hashtags, solo eliminar el símbolo (#)
    if(lineaLeida<=log):
        print('--Retira hashtags')
    texto=re.sub("[#]","",texto)
    #Eliminar correos electrónicos.
    if(lineaLeida<=log):
        print('--Elimina correos electronicos')
    texto= re.sub("(\S+@\S+\.\S)", "", texto)
    
    # Separación de palabras en notación Camell: “QuedateEnCasa→ Quedate En Casa”.
    if(lineaLeida<=log):
        print('-- Separa palabras con notacion Camell')
    patron=re.compile('([ÁÉÍÓÚA-Z]*[a-z0-9]+[ÁÉÍÓÚA-Z]+)+')
    for palabra in texto.split(' '):
        if patron.match(palabra)!=None:
            palabraSep = " ".join(re.findall('[ÁÉÍÓÚA-Z]+[a-z0-9]*',palabra))
            texto=re.sub(palabra,palabraSep,texto)
            if(lineaLeida<=log):
                print('       ---- Palabra encontrada: ' + palabraSep)
            cammel=+1
    if(cammel<log):
        if(lineaLeida<=log):
            print('       ---- Ninguna palabra encontrada')
    cammel=0
    
    
    #Extraer/listar todas las frases que inicien con la preposición “en”, seguido de espacio en blanco y luego una palabra que inicie en Mayúsculas.
    if(lineaLeida<=log):
        print('-- Lista frases que inicias con "en" ')
        print('       ---- Frase encontrada' + str(re.findall('\sen\s[A-Z]\S+',texto)))
    
    #Normalizar todo a minúsculas.
    if(lineaLeida<=log):
        print('-- Normalizar a minusculas')
    
    texto = texto.lower()
    
    if(lineaLeida<=log):
        print('       ---- '+ texto)
    
    #Eliminar palabras vacías.
    if(lineaLeida<=log):
        print('-- Elimina palabras vacías')
        
    texto=" ".join([palabra for palabra in texto.split(' ') if palabra not in palabras_vacias])
    
    if(lineaLeida<=log):
        print('       ---- '+ texto)
    
    #Lematizar
    if(lineaLeida<=log):
        print('-- Lematiza')
    
    textol=lematiza(texto)
    
    for palabra in textol:
        lema= palabra.lemma_
        palabra_original=palabra.text
        cadenalema=lema+','+cadenalema
        if(lineaLeida<=log):
            print('       ---- '+palabra_original+'-->'+lema)
    
    if(lineaLeida<=log):
        print('       -->'+cadenalema)
    
    #------------------------------------------------------------
    df_nuevafila = pd.DataFrame({'texto': [cadenalema]})
    df_lema = pd.concat([df_lema, df_nuevafila], ignore_index=True)
    cadenalema=''
    #------------------------------------------------------------
            
    #Stemmer
    if(lineaLeida<=log):
        print('-- Stemmer - Raíz de la palabra')
    s=word_tokenize(texto)
    esp=SnowballStemmer('spanish')
    text_r=[esp.stem(token) for token in s]
    str_text_r = ','.join(text_r)    
        
    if(lineaLeida<=log):
        print('       ---- '+str(str_text_r))
    
    #------------------------------------------------------------
    df_nuevafila = pd.DataFrame({'texto': [str_text_r]})
    df_Stemmer = pd.concat([df_Stemmer, df_nuevafila], ignore_index=True)
    str_text_r=''
    #------------------------------------------------------------
        
print("Se procesaron: " + str(lineaLeida))

---------------------------------- Linea 1 ---------------------------------- 
--Elimina @user
--Elimina caracteres especiales [\.$\*¿\?¡!\+\-\[\]\)\(\\]"," 
--Retira hashtags
--Elimina correos electronicos
-- Separa palabras con notacion Camell
       ---- Ninguna palabra encontrada
-- Lista frases que inicias con "en" 
       ---- Frase encontrada[]
-- Normalizar a minusculas
       ----  jajajaja dale, hacete la boluda vos jajaja igual a vos nunca se te puede tomar en serio te mando un abrazo desde perú
-- Elimina palabras vacías
       ----  jajajaja dale, hacete boluda vos jajaja igual vos nunca puede tomar serio mando abrazo perú
-- Lematiza
       ----  --> 
       ---- jajajaja-->jajajaja
       ---- dale-->dale
       ---- ,-->,
       ---- hacete-->hacetar
       ---- boluda-->boluda
       ---- vos-->vo
       ---- jajaja-->jajaja
       ---- igual-->igual
       ---- vos-->vo
       ---- nunca-->nunca
       ---- puede-->poder
       ---- tomar-->tomar
       ---- serio-->s

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
# Crear un objeto TfidfVectorizer
vectorizerLema = TfidfVectorizer()
vectorizerStem = TfidfVectorizer()

# Ajustar el vectorizador en los datos de entrada
vectorizerLema.fit(df_lema['texto'])
vectorizerStem.fit(df_Stemmer['texto'])

# Transformar los datos de entrada en la matriz TF-IDF
tfidf_matrixLema = vectorizerLema.transform(df_lema['texto'])
tfidf_matrixStem = vectorizerStem.transform(df_Stemmer['texto'])

In [11]:
# Obtener el diccionario de términos del vectorizador
terms_dictLema = vectorizerLema.vocabulary_
terms_dictStem = vectorizerStem.vocabulary_

# Obtener los nombres de los términos a partir del diccionario de términos
feature_namesLema = [term for term, index in sorted(terms_dictLema.items(), key=lambda x: x[1])]
feature_namesStem = [term for term, index in sorted(terms_dictStem.items(), key=lambda x: x[1])]

# Convertir la matriz de TF-IDF en un DataFrame de pandas
tfidf_dfLema = pd.DataFrame(tfidf_matrixLema.todense(), columns=feature_namesLema)
tfidf_dfStem = pd.DataFrame(tfidf_matrixStem.todense(), columns=feature_namesStem)

In [ ]:
tfidf_dfLema.head(3)

In [ ]:
tfidf_dfStem.head(3)

In [12]:
#agregar columna de id a los 2 datasets tfidf
tfidf_dfLema['Indice']=tfidf_dfLema.index
tfidf_dfStem['Indice']=tfidf_dfStem.index

#agregar la columna de id al dataset que contiene las etiquetas
#cambiamos el nombre de etiqueta para evitar duplicado con otra columna
df_Cardi['Indice']=df_Cardi.index
df_Cardi['Trg']=df_Cardi['etiqueta']

In [14]:
#hacer join/merge con tfidf Lema y el dataset original que tiene las etiquetas
#Los campos en lema son en minusculas y el campo Indice con I mayuscula
tfidf_dfLema=pd.merge(tfidf_dfLema,df_Cardi[['Trg','Indice']], on='Indice')
tfidf_dfLema.head(3)

,00,000,01,02,03,04,10,100,1000,10000,...,ñoño,órgano,últimamente,último,últimodíadenavidad,único,úselo,útil,Indice,Trg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2


In [15]:
#hacer join/merge con tfidf Stemmer y el dataset original que tiene las etiquetas
#Los campos en lema son en minusculas y el campo Indice con I mayuscula
tfidf_dfStem=pd.merge(tfidf_dfStem,df_Cardi[['Trg','Indice']], on='Indice')
tfidf_dfStem.head(3)

,00,000,01,02,03,04,10,100,1000,10000,...,zonz,zoo,zorr,zum,zumbadisim,zurd,ño,ñoñ,Indice,Trg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2


In [16]:
# crea el archivo Lema TF-IDF
tfidf_dfLema.to_csv('tfidf_dfLema.csv',index=False)
# crea el archivo Stemmer TF-IDF
tfidf_dfStem.to_csv('tfidf_dfStem.csv',index=False)